In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

# Read the CSV and Perform Basic Data Cleaning

In [4]:
# https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-

columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [6]:
# Load the data
file_path = Path('../Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [7]:
#Encode the data

loans_binary_encoded = pd.get_dummies(df, columns=["home_ownership","verification_status","next_pymnt_d","initial_list_status","pymnt_plan","issue_d", "application_type","debt_settlement_flag", "hardship_flag"])
loans_binary_encoded.head()

,loan_amnt,int_rate,installment,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,initial_list_status_f,initial_list_status_w,pymnt_plan_n,issue_d_Feb-2019,issue_d_Jan-2019,issue_d_Mar-2019,application_type_Individual,application_type_Joint App,debt_settlement_flag_N,hardship_flag_N
0,10500.0,0.1719,375.35,66000.0,low_risk,27.24,0.0,0.0,8.0,0.0,...,0,1,1,0,0,1,1,0,1,1
1,25000.0,0.2000,929.09,105000.0,low_risk,20.23,0.0,0.0,17.0,1.0,...,0,1,1,0,0,1,1,0,1,1
2,20000.0,0.2000,529.88,56000.0,low_risk,24.26,0.0,0.0,8.0,0.0,...,0,1,1,0,0,1,1,0,1,1
3,10000.0,0.1640,353.55,92000.0,low_risk,31.44,0.0,1.0,10.0,1.0,...,0,1,1,0,0,1,1,0,1,1
4,22000.0,0.1474,520.39,52000.0,low_risk,18.76,0.0,1.0,14.0,0.0,...,0,1,1,0,0,1,1,0,1,1


# Split the Data into Training and Testing

In [8]:
# Create our features
X = loans_binary_encoded.copy(columns)
X = X.drop(columns="loan_status")

# Create our target
y = loans_binary_encoded.copy()
y = y.loan_status

In [9]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,initial_list_status_f,initial_list_status_w,pymnt_plan_n,issue_d_Feb-2019,issue_d_Jan-2019,issue_d_Mar-2019,application_type_Individual,application_type_Joint App,debt_settlement_flag_N,hardship_flag_N
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.123879,0.876121,1.0,0.371696,0.451066,0.177238,0.860340,0.139660,1.0,1.0
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.329446,0.329446,0.0,0.483261,0.497603,0.381873,0.346637,0.346637,0.0,0.0
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,1.000000,1.0,0.000000,0.000000,0.000000,1.000000,0.000000,1.0,1.0
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,1.000000,1.0,0.000000,0.000000,0.000000,1.000000,0.000000,1.0,1.0
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,0.000000,1.000000,1.0,1.000000,1.000000,0.000000,1.000000,0.000000,1.0,1.0
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0


In [10]:
# Check the balance of our target values
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [51]:
# Split the X and y into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)
X_train.shape

(51612, 95)

# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [52]:
# Resample the training data with the RandomOversampler
from imblearn.ensemble import BalancedRandomForestClassifier
model = BalancedRandomForestClassifier(n_estimators=100, random_state=0)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)


In [53]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

0.7909802547026521

In [54]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[   58,    29],
       [ 1450, 15668]])

In [55]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.04      0.67      0.92      0.07      0.78      0.60        87
   low_risk       1.00      0.92      0.67      0.95      0.78      0.63     17118

avg / total       0.99      0.91      0.67      0.95      0.78      0.63     17205



In [56]:
# List the features sorted in descending order by feature importance
sorted(zip(model.feature_importances_, X.columns), reverse=True)

[(0.08945744937550396, 'total_rec_prncp'),
 (0.06596254144090204, 'total_pymnt_inv'),
 (0.059335570067269396, 'total_rec_int'),
 (0.05790438749472208, 'total_pymnt'),
 (0.054349457744250755, 'last_pymnt_amnt'),
 (0.03112476570430989, 'int_rate'),
 (0.01826990018366979, 'issue_d_Jan-2019'),
 (0.018220121000255652, 'revol_bal'),
 (0.018021425582499667, 'annual_inc'),
 (0.01757468344434905, 'total_bc_limit'),
 (0.017522508829635488, 'out_prncp'),
 (0.01737231565926795, 'dti'),
 (0.016116380620556987, 'installment'),
 (0.015904492137130546, 'max_bal_bc'),
 (0.015842044097941127, 'avg_cur_bal'),
 (0.015054025048725975, 'mo_sin_old_il_acct'),
 (0.014933757743439052, 'mths_since_recent_inq'),
 (0.014525333464667979, 'mo_sin_old_rev_tl_op'),
 (0.014144246580480345, 'issue_d_Mar-2019'),
 (0.014092067305320432, 'il_util'),
 (0.014031996643172073, 'tot_hi_cred_lim'),
 (0.013639191065311581, 'out_prncp_inv'),
 (0.013289427904458329, 'bc_util'),
 (0.01325493084179471, 'bc_open_to_buy'),
 (0.0126484

### Easy Ensemble AdaBoost Classifier

In [57]:
# Train the Classifier
from imblearn.ensemble import EasyEnsembleClassifier
eec = EasyEnsembleClassifier(n_estimators=100,random_state=42)
eec.fit(X_train, y_train)

y_pred = eec.predict(X_test)

In [58]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_pred)

0.9261575836687335

In [59]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[   79,     8],
       [  954, 16164]])

In [60]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.08      0.91      0.94      0.14      0.93      0.85        87
   low_risk       1.00      0.94      0.91      0.97      0.93      0.86     17118

avg / total       0.99      0.94      0.91      0.97      0.93      0.86     17205



In [ ]:
# The Balanced Random Forest Classifier Analysis

# 1. Balanced Accuracy Score for this was 0.79, which is good.
# 2. The Recall(sensitivity) of "high_risk" is 0.67, means 67% of high risk cases were found correctly. 
#    This is a decent score.
# 3. The Recall of "low_risk" is 0.92, means 92% of the low risk cases were found correctly. 
#    Again this is a decent score.
# 4. The Precision of "high risk" cases is 0.04, means 4% of high risk cases were recorded. 
# 5. The precision of "low risk" cases is 1.00, means  100% of low risk cases were recorded.
# 6. The top 5 features with most importance are 'total_rec_prncp','total_pymnt_inv','total_rec_int','total_pymnt'
# 'last_pymnt_amnt

# Easy Ensemble AdaBoost Classifier

# 1. Balanced Accuracy Score for this was 0.92, which is a great score.
# 2. The Recall(sensitivity) of "high_risk" is 0.91, means 91% of high risk cases were found correctly. 
# 3. The Recall of "low_risk" is 0.94, means 94% of the low risk cases were found correctly. 
# 4. The Precision of "high risk" cases is 0.08, means 8% of high risk cases were recorded. 
# 5. The precision of "low risk" cases is 1.00, means 100% of low risk cases were recorded. 


#I would recommend Easy Ensemble AdaBoost Classifier, beacause:
# 1. The number of high risk cases found correctly were 92% which is really helpful for the credit risk company,
# so that they wont give away to high risk cases.